# Sentiment analysis of show reviews

The goal of this analysis is to ...
* Get hands-on experience with packages and tools for analysing Russian language (natasha, nltk, spacy, rnnmorph, pymorphy2)
* Investigate available pre-trained models for Russian language (wor2vec, fasttext, navec, models from sber, deeppavlov and others)
* Learn how to finetune BERT-like models

## Imports

In [30]:
# !pip install dateparser

In [1]:
import os
import sys
import warnings

import dateparser
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier, Pool, cv, metrics
from matplotlib import pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.exceptions import ConvergenceWarning
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
    RobustScaler,
    StandardScaler,
)
from tqdm import tqdm

tqdm.pandas()

optuna.logging.set_verbosity(optuna.logging.WARNING)
sns.set(rc={"figure.dpi": 100, "savefig.dpi": 300})
sns.set_context("notebook")
sns.set_style("ticks")

ConvergenceWarning("ignore")

warnings.filterwarnings("ignore")
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ[
        "PYTHONWARNINGS"
    ] = "ignore::UserWarning,ignore::ConvergenceWarning,ignore::RuntimeWarning"
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
SEED = 42

## Data

### Loading data

In [3]:
movie_reviews_df = pd.read_parquet("../../data/anonymized_movies_reviews.parquet")
movie_reviews_df.shape

(171094, 7)

In [4]:
series_reviews_df = pd.read_parquet("../../data/anonymized_series_reviews.parquet")
series_reviews_df.shape

(35643, 7)

### Dataset overview

In [5]:
movie_reviews_df.sample(n=10, random_state=SEED)

,show_id,user_id,datetime,sentiment,subtitle,review_body,usefulness_ratio
29900,1965,44279,21 января 2010 19:09,good,,"Юг США, Миссисипи, отличающееся расовым разноо...",16 / 4
154753,977743,49610,12 ноября 2018 00:10,bad,"'Форма воды', Оскар, отношение, размышления, а...",В последний раз я столько хмурила брови и подн...,5 / 3
161478,623250,59545,05 марта 2018 14:59,good,Новая глава любимой саги,Мне хочется сравнить этот фильм с элегантным и...,20 / 39
73900,49166,33188,01 мая 2014 22:18,good,Маскарад абсурда,Для меня мультфильм 'Коралина в Стране Кошмаро...,1 / 1
39455,81733,46751,19 февраля 2009 14:39,bad,Из грязи - в князи?,"Я люблю звенящую прозрачную Киру Найтли, с ее ...",25 / 32
91332,450765,37060,14 марта 2011 18:35,neutral,Любовь не по плану,А вы когда-нибудь задумывались: а что если все...,11 / 7
38957,81733,30715,21 апреля 2018 23:07,good,,Как же приятно смотреть картины где нет ничего...,3 / 7
129787,6144,34494,05 января 2011 13:55,good,,"Пожалуй, один из тех фильмов, которые будут 'в...",7 / 3
79575,705356,36252,22 января 2015 02:06,good,Есть лишь любовь и есть страх,Очень много шумихи поднял совершенно недавно в...,6 / 7
119379,649917,22210,09 октября 2018 09:23,good,"Следуй Обету, данному тобой","Потрясающий фильм о скромном, милом, добром мо...",3 / 0


In [6]:
series_reviews_df.sample(n=10, random_state=SEED)

,show_id,user_id,datetime,sentiment,subtitle,review_body,usefulness_ratio
17123,838050,26321,03 апреля 2016 20:22,neutral,А был ли метод?,Вот и я сподобилась посмотреть нашумевший сери...,7 / 6
29947,306084,21093,20 марта 2010 20:23,good,- Почему он не разговаривает? - Ботан же. ©,"Наконец-то я для себя нашел любимый сериал, ко...",57 / 14
14526,1445244,56442,14 сентября 2021 12:51,bad,Первый блин комом,Единственный плюс этого сериала - отличная опе...,65 / 25
8471,453191,68606,26 декабря 2014 18:15,bad,«Санта-Барбара» по-вампирски.,А как неплохо все начиналось… не занудно… акте...,11 / 7
5288,38898,38121,18 июня 2010 00:25,good,,"Вот оттого, что книгу не читал, фильм и не пон...",29 / 17
17419,1331277,7511,19 января 2021 12:13,good,"Если человек обещает что-то, что заведомо не с...",Хорошая фраза и неплохой сериал.<p>Зацепила ид...,12 / 2
1043,401152,6398,11 января 2017 22:28,good,В адаптации телеканала HBO.,Часть первая. <p>Лет пять назад я смотрел сие ...,3 / 4
30271,1209845,38603,13 марта 2021 02:48,good,Достойное продолжение франшизы,Первый фильм был очень популярен. Но к сожален...,14 / 6
33923,502838,60726,20 июня 2015 02:49,neutral,Больше Шерлоков хороших и разных..,Пролог.<p>Этот популярный нынче сериал я посмо...,3 / 6
17767,585100,60714,11 января 2016 15:29,good,,"Изредка я писал рецензии, это большая для меня...",13 / 1


Let's merge datasets

In [7]:
movie_reviews_df.insert(2, "type", "movie")
series_reviews_df.insert(2, "type", "movie")
reviews_df = pd.concat([movie_reviews_df, series_reviews_df]).reset_index(drop=True)

In [8]:
reviews_df.sample(n=10, random_state=SEED)

,show_id,user_id,type,datetime,sentiment,subtitle,review_body,usefulness_ratio
196236,257386,28525,movie,27 августа 2010 11:23,good,Вечный город.,К истории Древнего мира у меня отношение особо...,50 / 10
128582,688832,44953,movie,20 февраля 2015 23:19,bad,50 оттенков разочарования,"Говорю сразу, книги читала все, да и по нескол...",11 / 6
159673,349,33910,movie,15 апреля 2018 21:04,good,"Господи, спасибо, что не пронесло мимо","Есть два типа фильмов, мой друг. Одни ты прост...",5 / 1
109244,686898,44065,movie,11 сентября 2019 11:58,neutral,Что же стало с клоуном?,"Итак, в первую очередь хотелось бы отметить то...",5 / 3
92610,61455,66782,movie,27 ноября 2017 18:52,good,Они отказываются подчиняться,"Автора этого замечательного фильма, Джосса Уэд...",5 / 0
43727,491724,44563,movie,28 января 2012 23:18,good,Жестокая правда,"Финчер снова нас поразил, он всегда нас поража...",18 / 11
50195,102130,67145,movie,17 августа 2009 11:56,good,"Преодолеть 2 года жизни, что встретиться","Слышал о фильме много, и в основном положитель...",14 / 3
43651,491724,66283,movie,19 февраля 2012 15:33,bad,"Мужчины, которые ненавидели женщин.",Я попробовала рассматривать этот фильм с двух ...,20 / 17
48775,7226,7905,movie,20 февраля 2014 03:44,good,,"«Догвилль» - это один из тех редких фильмов, п...",5 / 2
75262,458,33255,movie,22 июня 2013 21:14,good,Тайна закрытой двери,Я имела счастье смотреть этот мультфильм в кин...,6 / 0


### Dataset Close Look

In this section, let's look at what columns we have, what type they are, how many non-zero values, etc.

But before, let's replace empty strings with NaN values.

In [9]:
reviews_df.replace(r"^\s*$", np.nan, regex=True, inplace=True)

In [10]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206737 entries, 0 to 206736
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   show_id           206737 non-null  int64 
 1   user_id           206737 non-null  int64 
 2   type              206737 non-null  object
 3   datetime          206737 non-null  object
 4   sentiment         206737 non-null  object
 5   subtitle          173836 non-null  object
 6   review_body       206737 non-null  object
 7   usefulness_ratio  206737 non-null  object
dtypes: int64(2), object(6)
memory usage: 12.6+ MB


As we can see, `subtitle` is empty sometimes.

Based on this look I've determined some things to do:
* Look at the columns, determine the number of unique values, duplicates, etc.
* **[+]** Normalize `datetime` column 
* Calculate `usefulness_ratio` based on the format `likes / dislikes`

## EDA

### `datetime`

Currently, datetime looks like this:

In [11]:
reviews_df["datetime"][0]

'16 февраля 2022 11:21'

I could create map with months manually, replace month for every date and then change the format, but I will use `dateparser` package 

In [12]:
dateparser.parse(reviews_df["datetime"][0]).strftime("%d.%m.%Y %H:%M")

'16.02.2022 11:21'

In [14]:
def normalize_datetime(datetime: str) -> str:
    return dateparser.parse(datetime).strftime("%d.%m.%Y %H:%M")

In [17]:
reviews_df["datetime"] = reviews_df["datetime"].progress_apply(normalize_datetime)

100%|█████████████████████████████████████████████████████████████████████████| 206737/206737 [04:56<00:00, 696.29it/s]


### `usefulness_ratio`

Currently, `usefulness_ratio` looks like this:

In [20]:
reviews_df["usefulness_ratio"].head(20)

0      0 / 1
1      2 / 5
2      2 / 3
3     6 / 16
4      4 / 6
5      3 / 6
6      6 / 6
7      8 / 6
8     7 / 67
9      1 / 9
10     4 / 6
11     8 / 0
12     3 / 7
13     4 / 1
14     1 / 1
15     9 / 1
16     4 / 0
17     2 / 0
18     3 / 1
19     2 / 1
Name: usefulness_ratio, dtype: object